In [ ]:
from data_cleaning_functions import cleaning_pipeline
from information import create_info_object
from machine_learning_functions import AnomalyDetector
from machine_learning_functions import ADWIN_drift

In [ ]:
def get_datapoint():
    # In some manner receives data point as a dictionary of form
    ''' datapoint={timestamp: datetime, 
        isset_id: string,
        name: string,
        kpi: string,
        operation: string,
        sum: float
        avg: float,
        min: float,
        max: float,
        var: float}'''
    #for example

    datapoint = {
    'timestamp': 'timepoint',
    'isset_id': 'ast-yhccl1zjue2t',
    'name': 'metal_cutting',
    'kpi': 'time',
    'operation': 'working',
    'sum': float, 
    'avg': float,
    'min': float,
    'max': float,
    'var': float}
    
    return datapoint


In [ ]:
#initializing the info manager
im = create_info_object()

In [ ]:

while True: #loops continuosly

    #first we call get_datapoint and we wait for a new input to arrive
    new_datapoint = get_datapoint()

    #once the new data point is aquired we clean it
    cleaned_datapoint = cleaning_pipeline(new_datapoint, im)

    #we now check if some drift has been detected
    drift_flag=ADWIN_drift(cleaned_datapoint, im)

    #we call the database to extract historical data
    historical_data = get_historical_data()


    

    